## **Renta mitjana**

Aquest dataset de l'Instituto Nacional d'Estadística inclou estadístiques relacionades amb la renta per secció censal.

* Font: INE
* Nom: Indicadores de renta media y mediana
* URL: https://www.ine.es/jaxiT3/Tabla.htm?t=30896&L=0
* Periodicitat: anual (amb dos anys de retràs)
* Agregació geofràfica: secció censal

De la URL, cal descarregar tots els indicadors agregats per secció, i seleccionant tots els anys possibles.

In [4]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import numpy as np
import pandas as pd
import seaborn as sns
from shapely import wkt
import json
import datetime
import csv
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

ROOT ="/content/drive/Shareddrives/GESTIÓ DE PROJECTES/X. Entregable"
RENTA_MITJA_CSV = ROOT + "/cleaning/raw_data/renta_mitja_ine.csv"

FROM_YEAR = 2018 # (included)
TO_YEAR = 2022 # (included)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
rentaMedia_INE = pd.read_csv(RENTA_MITJA_CSV, delimiter=';').dropna()

MUNICIPIOS_INE_LIST = ["08019 Barcelona", "08101 Hospitalet de Llobregat, L'", "08089 Gavà", "08194 Sant Adrià de Besòs", "08211 Sant Feliu de Llobregat", "08301 Viladecans"]
rentaMedia_INE = rentaMedia_INE[rentaMedia_INE['Municipios'].isin(MUNICIPIOS_INE_LIST)]

rentaMedia_INE = rentaMedia_INE.drop(columns=['Municipios', "Distritos"])
rentaMedia_INE = pd.pivot_table(rentaMedia_INE, values='Total', index=['Periodo', 'Secciones'], columns=['Indicadores de renta media y mediana']).reset_index().rename_axis(None, axis=1)

rentaMedia_INE['CUSEC'] = rentaMedia_INE['Secciones'].str[:10]
rentaMedia_INE.columns = [x.strip() for x in rentaMedia_INE.columns]
rentaMedia_INE["Media de la renta por unidad de consumo"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000
rentaMedia_INE["Mediana de la renta por unidad de consumo"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000
rentaMedia_INE["Renta bruta media por hogar"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000
rentaMedia_INE["Renta bruta media por persona"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000
rentaMedia_INE["Renta mediana por hogar"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000
rentaMedia_INE["Renta neta media por hogar"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000
rentaMedia_INE["Renta neta media por persona"] = rentaMedia_INE["Renta neta media por persona"].astype(float) * 1000

rentaMedia_INE = rentaMedia_INE.drop(columns="Secciones")
rentaMedia_INE.head()

,Periodo,Media de la renta por unidad de consumo,Mediana de la renta por unidad de consumo,Renta bruta media por hogar,Renta bruta media por persona,Renta mediana por hogar,Renta neta media por hogar,Renta neta media por persona,CUSEC
0,2015,9310.0,9310.0,9310.0,9310.0,9310.0,9310.0,9310.0,0801901001
1,2015,7496.0,7496.0,7496.0,7496.0,7496.0,7496.0,7496.0,0801901002
2,2015,7300.0,7300.0,7300.0,7300.0,7300.0,7300.0,7300.0,0801901003
3,2015,9346.0,9346.0,9346.0,9346.0,9346.0,9346.0,9346.0,0801901004
4,2015,7690.0,7690.0,7690.0,7690.0,7690.0,7690.0,7690.0,0801901005


In [10]:
# OMPLIR ELS ANYS PELS QUE NO TENIM DADES AMB LES DADES DE L'ANY MÉS PROPER
for year in range(FROM_YEAR, TO_YEAR+1):
    if year not in rentaMedia_INE["Periodo"].unique() and year>rentaMedia_INE["Periodo"].min():
        print("missing",year,"-> copying from",rentaMedia_INE["Periodo"].max())
        last_year = rentaMedia_INE["Periodo"].max()
        last_year_data = rentaMedia_INE[rentaMedia_INE['Periodo'] == last_year].copy()
        last_year_data.loc[:,"Periodo"] = last_year_data.loc[:,"Periodo"].apply(lambda x: year)
        rentaMedia_INE = pd.concat([rentaMedia_INE, last_year_data], ignore_index=True)

missing 2022 -> copying from 2021


In [8]:
# EXPORTAR
CLEAN_DATA_PATH = ROOT + "/cleaning/clean_data/"
RENTA_MEDIA_OUTPUT_FILE = "renta_media_ine_clean.csv"

rentaMedia_INE.to_csv(CLEAN_DATA_PATH + RENTA_MEDIA_OUTPUT_FILE, index=False)